# Data Cleaning Project

A data cleaning project from Iron Hack's Data Analysis bootcamp.

In [1]:
# libraries needed

# %pip install pandas
# %pip install numpy
# %pip install warning
# %pip install regex
# %pip install warnings

In [2]:
# Libraries
import pandas as pd
pd.set_option('display.max_columns', None)

import numpy as np

import warnings
warnings.filterwarnings('ignore')

from fuzzywuzzy import fuzz

In [3]:
# Reading csv. The original df is not going to be modified for later comparison

df_ori = pd.read_csv('../src/attacks.csv', encoding='latin1')

df = df_ori.copy()

## DataFrame reading

In [4]:
df.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [5]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25723 entries, 0 to 25722
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case Number             8702 non-null   object 
 1   Date                    6302 non-null   object 
 2   Year                    6300 non-null   float64
 3   Type                    6298 non-null   object 
 4   Country                 6252 non-null   object 
 5   Area                    5847 non-null   object 
 6   Location                5762 non-null   object 
 7   Activity                5758 non-null   object 
 8   Name                    6092 non-null   object 
 9   Sex                     5737 non-null   object 
 10  Age                     3471 non-null   object 
 11  Injury                  6274 non-null   object 
 12  Fatal (Y/N)             5763 non-null   object 
 13  Time                    2948 non-null   object 
 14  Species                 3464 non-null 

In [6]:
df.shape

(25723, 24)

In [7]:
df.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

### Normalization of column names

In [8]:
# list of original col names
ori_names = list(df.columns)

ori_names[:5]

['Case Number', 'Date', 'Year', 'Type', 'Country']

In [9]:
ori_names[-4]

'Case Number.2'

In [10]:
# first general modification of columns
better_names = []

for c in ori_names:
    splitted = c.split()
    res = '_'.join(splitted).lower()
    better_names.append(res)
    
better_names[:5]

['case_number', 'date', 'year', 'type', 'country']

In [11]:
# change 'fatal_(y/n)' to just fatal, the '.' in 'case_number.x' to '_' and the unnamed columns to unnamed_1 and _2
better_names[12] = 'fatal'

better_names[-5] = better_names[-5].replace('.', '_')
better_names[-4] = better_names[-4].replace('.', '_')

better_names[-2] = 'unnamed_1'
better_names[-1] = 'unnamed_2'

better_names

['case_number',
 'date',
 'year',
 'type',
 'country',
 'area',
 'location',
 'activity',
 'name',
 'sex',
 'age',
 'injury',
 'fatal',
 'time',
 'species',
 'investigator_or_source',
 'pdf',
 'href_formula',
 'href',
 'case_number_1',
 'case_number_2',
 'original_order',
 'unnamed_1',
 'unnamed_2']

In [12]:
better_names[-5]

'case_number_1'

In [13]:
df.columns = better_names

df.head()

,case_number,date,year,type,country,area,location,activity,name,sex,age,injury,fatal,time,species,investigator_or_source,pdf,href_formula,href,case_number_1,case_number_2,original_order,unnamed_1,unnamed_2
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


### Rearranging columns

case_number 1 and 2 are duplicates of case_number. Since we cannot drop columns, I'm going to change it to 0 and keep it for later use if I need an extra column.


In fact, the order of the columns is weird, so I'm first going to fill the columns I named with 0, change their name to unnamed_3 and 4 and then re-arrange.

In [14]:
new_col_order = [
                'case_number', # the 'ID' of the attack
                'date', 'year', 'time', 'country', 'area', 'location', # time and place
                'name', 'sex', 'age', # who
                'type','activity', 'injury', 'fatal', # injury
                'species', # the only shark data, makes sense to have the shark data after the injury data
                'original_order','investigator_or_source', 'pdf', 'href_formula', 'href', # investigation data
                'unnamed_1', 'unnamed_2', 'case_number_1', 'case_number_2' # the empty cols
                ]

df = df[new_col_order]

In [15]:
df.loc[:, ['case_number_1', 'case_number_2']] = 0


## Nul Values

In [16]:
def napercent():
    result = df.isna().mean().sort_values(ascending=False) * 100
    display(result[result > 0])
    
napercent()

unnamed_1                 99.996112
unnamed_2                 99.992225
time                      88.539439
species                   86.533453
age                       86.506240
sex                       77.697003
activity                  77.615364
location                  77.599813
fatal                     77.595926
area                      77.269370
name                      76.316915
country                   75.694903
injury                    75.609377
investigator_or_source    75.566614
type                      75.516075
year                      75.508300
href_formula              75.504412
date                      75.500525
pdf                       75.500525
href                      75.500525
original_order            75.473312
case_number               66.170353
dtype: float64

In [17]:
nan_cols = df.isna().sum()

nan_cols[nan_cols>0]

case_number               17021
date                      19421
year                      19423
time                      22775
country                   19471
area                      19876
location                  19961
name                      19631
sex                       19986
age                       22252
type                      19425
activity                  19965
injury                    19449
fatal                     19960
species                   22259
original_order            19414
investigator_or_source    19438
pdf                       19421
href_formula              19422
href                      19421
unnamed_1                 25722
unnamed_2                 25721
dtype: int64

In [18]:
#drop all columns with half or more the columns with nan values
df.dropna(thresh=len(df.columns) / 2, inplace=True)

In [19]:
df.shape, df_ori.shape

((6302, 24), (25723, 24))

In [20]:
napercent()

unnamed_1                 99.984132
unnamed_2                 99.968264
time                      53.221200
species                   45.033323
age                       44.922247
sex                        8.965408
activity                   8.632180
location                   8.568708
fatal                      8.552840
area                       7.219930
name                       3.332275
country                    0.793399
injury                     0.444303
investigator_or_source     0.269756
type                       0.063472
year                       0.031736
href_formula               0.015868
case_number                0.015868
dtype: float64

### Filling unnamed columns with 0s

In [21]:
# there are 3 values in this columns, they seem to be comments on the row, but without context they add no value, so I'm going to replace them with 0
df.loc[:, ['unnamed_1', 'unnamed_2']] = 0

### Checking specific columns

In [22]:
napercent()

time                      53.221200
species                   45.033323
age                       44.922247
sex                        8.965408
activity                   8.632180
location                   8.568708
fatal                      8.552840
area                       7.219930
name                       3.332275
country                    0.793399
injury                     0.444303
investigator_or_source     0.269756
type                       0.063472
year                       0.031736
href_formula               0.015868
case_number                0.015868
dtype: float64

In [23]:
# These are unknown, because we only know when it was reported
df[df['year'].isna() == True]

,case_number,date,year,time,country,area,location,name,sex,age,type,activity,injury,fatal,species,original_order,investigator_or_source,pdf,href_formula,href,unnamed_1,unnamed_2,case_number_1,case_number_2
187,2017.01.08.R,Reported 08-Jan-2017,NaN,NaN,AUSTRALIA,Queensland,NaN,Kerry Daniel,M,35,Invalid,Spearfishing,"No attack, shark made a threat display",NaN,Bull shark,6116.0,Liquid Vision 1/8/2017,2017.01.08.R-KerryDaniel.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,0,0,0,0
6079,1836.08.19.R,Reported 19-Aug-1836,NaN,NaN,ENGLAND,Cumberland,Whitehaven,a boy,M,NaN,Unprovoked,Swimming,FATAL,Y,NaN,224.0,"C. Moore, GSAF",1835.08.19.R-Whitehaven.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,0,0,0,0


### Last Columns

Hypothesis: the columns with nan are most probably unknown values

In [24]:
df.fillna('unknown', inplace=True)

In [25]:
napercent()

Series([], dtype: float64)

## Check incorrect values

### Binary columns

In [26]:
# first sex
df.sex.value_counts()

M          5094
F           637
unknown     565
M             2
N             2
lli           1
.             1
Name: sex, dtype: int64

In [27]:
df[df.sex == '.']

,case_number,date,year,time,country,area,location,name,sex,age,type,activity,injury,fatal,species,original_order,investigator_or_source,pdf,href_formula,href,unnamed_1,unnamed_2,case_number_1,case_number_2
5437,1908.06.02.R,Reported 02-Jun-1908,1908.0,unknown,PAPUA NEW GUINEA,New Britain,Matupi,unknown,.,unknown,Sea Disaster,.,"Remains of 3 humans recovered from shark, but ...",Y,Allegedly a 33-foot shark,866.0,"Taranaki Herald, 6/2/1908",1908.06.02.R-Matupi.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,0,0,0,0


In [28]:
df.sex.unique()

array(['F', 'M', 'unknown', 'M ', 'lli', 'N', '.'], dtype=object)

In [29]:
# The 'N' might be a missinput of 'M', same with 'M '. The 'lli', according to the search, is Brian, a M. '.' is unknown.
df['sex'] = np.where((df['sex'] == 'M ') | (df['sex'] == 'N') | (df['sex'] == 'lli') | (df['sex'] == 'M'), 'M', np.where(df['sex'] == 'F', 'F', 'unknown'))

df.sex.value_counts()

M          5099
F           637
unknown     566
Name: sex, dtype: int64

In [30]:
# now with Fatal
df.fatal.value_counts()

N          4293
Y          1388
unknown     539
UNKNOWN      71
 N            7
M             1
2017          1
N             1
y             1
Name: fatal, dtype: int64

In [31]:
# Values should be 'Y', 'N' or 'Unknown'. I'm going to do the same as above but with another method
df.fatal = df.fatal.map(lambda x: 'unknown' if 'unknown' in x.lower() else ('Y' if 'y' in x.lower() else 'N' if 'n' in x.lower() or 'm' in x.lower() else 'unknown'))

df.fatal.value_counts()

N          4302
Y          1389
unknown     611
Name: fatal, dtype: int64

In [32]:
df.head()

,case_number,date,year,time,country,area,location,name,sex,age,type,activity,injury,fatal,species,original_order,investigator_or_source,pdf,href_formula,href,unnamed_1,unnamed_2,case_number_1,case_number_2
0,2018.06.25,25-Jun-2018,2018.0,18h00,USA,California,"Oceanside, San Diego County",Julie Wolfe,F,57,Boating,Paddling,"No injury to occupant, outrigger canoe and pad...",N,White shark,6303.0,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,0,0,0,0
1,2018.06.18,18-Jun-2018,2018.0,14h00 -15h00,USA,Georgia,"St. Simon Island, Glynn County",Adyson McNeely,F,11,Unprovoked,Standing,Minor injury to left thigh,N,unknown,6302.0,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,0,0,0,0
2,2018.06.09,09-Jun-2018,2018.0,07h45,USA,Hawaii,"Habush, Oahu",John Denges,M,48,Invalid,Surfing,Injury to left lower leg from surfboard skeg,N,unknown,6301.0,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,0,0,0,0
3,2018.06.08,08-Jun-2018,2018.0,unknown,AUSTRALIA,New South Wales,Arrawarra Headland,male,M,unknown,Unprovoked,Surfing,Minor injury to lower leg,N,2 m shark,6300.0,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,0,0,0,0
4,2018.06.04,04-Jun-2018,2018.0,unknown,MEXICO,Colima,La Ticla,Gustavo Ramos,M,unknown,Provoked,Free diving,Lacerations to leg & hand shark PROVOKED INCIDENT,N,"Tiger shark, 3m",6299.0,A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,0,0,0,0


### Fixing case_number


The three `case_number` had almost the same values, albeit some missinputs, so we're going to focus on the first one and use the other two later maybe for other type of information, as stated before.


In [33]:
df.sort_values('original_order')

,case_number,date,year,time,country,area,location,name,sex,age,type,activity,injury,fatal,species,original_order,investigator_or_source,pdf,href_formula,href,unnamed_1,unnamed_2,case_number_1,case_number_2
6301,ND.0001,1845-1853,0.0,unknown,CEYLON (SRI LANKA),Eastern Province,"Below the English fort, Trincomalee",male,M,15,Unprovoked,Swimming,"FATAL. ""Shark bit him in half, carrying away t...",Y,unknown,2.0,S.W. Baker,ND-0001-Ceylon.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,0,0,0,0
6300,ND.0002,1883-1889,0.0,unknown,PANAMA,unknown,"Panama Bay 8ºN, 79ºW",Jules Patterson,M,unknown,Unprovoked,unknown,FATAL,Y,unknown,3.0,"The Sun, 10/20/1938",ND-0002-JulesPatterson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,0,0,0,0
6299,ND.0003,1900-1905,0.0,unknown,USA,North Carolina,Ocracoke Inlet,Coast Guard personnel,M,unknown,Unprovoked,Swimming,FATAL,Y,unknown,4.0,"F. Schwartz, p.23; C. Creswell, GSAF",ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,0,0,0,0
6298,ND.0004,Before 1903,0.0,unknown,AUSTRALIA,Western Australia,unknown,Ahmun,M,unknown,Unprovoked,Pearl diving,FATAL,Y,unknown,5.0,"H. Taunton; N. Bartlett, pp. 233-234",ND-0004-Ahmun.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,0,0,0,0
6297,ND.0005,Before 1903,0.0,unknown,AUSTRALIA,Western Australia,Roebuck Bay,male,M,unknown,Unprovoked,Diving,FATAL,Y,unknown,6.0,"H. Taunton; N. Bartlett, p. 234",ND-0005-RoebuckBay.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,2018.06.04,04-Jun-2018,2018.0,unknown,MEXICO,Colima,La Ticla,Gustavo Ramos,M,unknown,Provoked,Free diving,Lacerations to leg & hand shark PROVOKED INCIDENT,N,"Tiger shark, 3m",6299.0,A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,0,0,0,0
3,2018.06.08,08-Jun-2018,2018.0,unknown,AUSTRALIA,New South Wales,Arrawarra Headland,male,M,unknown,Unprovoked,Surfing,Minor injury to lower leg,N,2 m shark,6300.0,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,0,0,0,0
2,2018.06.09,09-Jun-2018,2018.0,07h45,USA,Hawaii,"Habush, Oahu",John Denges,M,48,Invalid,Surfing,Injury to left lower leg from surfboard skeg,N,unknown,6301.0,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,0,0,0,0
1,2018.06.18,18-Jun-2018,2018.0,14h00 -15h00,USA,Georgia,"St. Simon Island, Glynn County",Adyson McNeely,F,11,Unprovoked,Standing,Minor injury to left thigh,N,unknown,6302.0,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,0,0,0,0


It seems that the `case_number` is related to the name of the pdf, so we're going to use that column as reference as it seems to be more reliable i.e. there are less duplicated values and 0 unknown.

In [34]:
df.pdf.value_counts()

1923.00.00.a-NJ fisherman.pdf      2
1935.06.05.R-SolomonIslands.pdf    2
1916.12.08.a-b-German.pdf          2
1907.10.16.R-HongKong.pdf          2
1921.11.27.a-b-Jack.pdf            2
                                  ..
1999.10.30.a-TroyJesse.pdf         1
1999.10.30.b-Dolsky.pdf            1
1999.11.00.a-Morson.pdf            1
1999.11.00.b-Bayo.pdf              1
ND-0001-Ceylon.pdf                 1
Name: pdf, Length: 6291, dtype: int64

In [35]:
df[df.pdf == 'unknown']

,case_number,date,year,time,country,area,location,name,sex,age,type,activity,injury,fatal,species,original_order,investigator_or_source,pdf,href_formula,href,unnamed_1,unnamed_2,case_number_1,case_number_2


In [36]:
# import regex to search for patterns
import re

def extract_case_number(row):

    pattern1 = r'(\d{4}\.\d{2}\.\d{2})' # yyyy.mm.dd pattern
    pattern2 = r'([a-zA-Z]{2}-\d{4})' # ND-XXXX pattern
    pattern3 = r'(\d{1,4}(?:-AD|-BC)?(?:AD|BC)?(?=[^\d]|$))' # XXXX-BC/AD or XXXXBC/AD pattern

    match = re.search(pattern1, row['pdf'])
    if match:
        return match.group(1)
    
    match = re.search(pattern2, row['pdf'])
    if match:
        return match.group(1)
    
    match = re.search(pattern3, row['pdf'])
    if match:
        return match.group(1)
    
    return None  # If the regex doesn't work, we can find na values to see why

df['case_number'] = df.apply(extract_case_number, axis=1)

df.tail(3)

,case_number,date,year,time,country,area,location,name,sex,age,type,activity,injury,fatal,species,original_order,investigator_or_source,pdf,href_formula,href,unnamed_1,unnamed_2,case_number_1,case_number_2
6299,ND-0003,1900-1905,0.0,unknown,USA,North Carolina,Ocracoke Inlet,Coast Guard personnel,M,unknown,Unprovoked,Swimming,FATAL,Y,unknown,4.0,"F. Schwartz, p.23; C. Creswell, GSAF",ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,0,0,0,0
6300,ND-0002,1883-1889,0.0,unknown,PANAMA,unknown,"Panama Bay 8ºN, 79ºW",Jules Patterson,M,unknown,Unprovoked,unknown,FATAL,Y,unknown,3.0,"The Sun, 10/20/1938",ND-0002-JulesPatterson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,0,0,0,0
6301,ND-0001,1845-1853,0.0,unknown,CEYLON (SRI LANKA),Eastern Province,"Below the English fort, Trincomalee",male,M,15,Unprovoked,Swimming,"FATAL. ""Shark bit him in half, carrying away t...",Y,unknown,2.0,S.W. Baker,ND-0001-Ceylon.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,0,0,0,0


In [37]:
napercent()
# there are not none values

Series([], dtype: float64)

### Fix year and original_order dtypes

In [38]:
df[df.year == 'unknown']

,case_number,date,year,time,country,area,location,name,sex,age,type,activity,injury,fatal,species,original_order,investigator_or_source,pdf,href_formula,href,unnamed_1,unnamed_2,case_number_1,case_number_2
187,2017.01.08,Reported 08-Jan-2017,unknown,unknown,AUSTRALIA,Queensland,unknown,Kerry Daniel,M,35,Invalid,Spearfishing,"No attack, shark made a threat display",unknown,Bull shark,6116.0,Liquid Vision 1/8/2017,2017.01.08.R-KerryDaniel.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,0,0,0,0
6079,1835.08.19,Reported 19-Aug-1836,unknown,unknown,ENGLAND,Cumberland,Whitehaven,a boy,M,unknown,Unprovoked,Swimming,FATAL,Y,unknown,224.0,"C. Moore, GSAF",1835.08.19.R-Whitehaven.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,0,0,0,0


In [39]:
df[df.original_order == 'unknown']

,case_number,date,year,time,country,area,location,name,sex,age,type,activity,injury,fatal,species,original_order,investigator_or_source,pdf,href_formula,href,unnamed_1,unnamed_2,case_number_1,case_number_2


No need to fix original order, but there are two rows in which year is `unknown`. I'm going to change it to the year of the report.

In [40]:
df.loc[187, 'year'] = 2017
df.loc[6079, 'year'] = 2017

Now, time to change everything to int.

In [41]:
df = df.astype({'year': 'int', 'original_order': 'int'})
df.head()

,case_number,date,year,time,country,area,location,name,sex,age,type,activity,injury,fatal,species,original_order,investigator_or_source,pdf,href_formula,href,unnamed_1,unnamed_2,case_number_1,case_number_2
0,2018.06.25,25-Jun-2018,2018,18h00,USA,California,"Oceanside, San Diego County",Julie Wolfe,F,57,Boating,Paddling,"No injury to occupant, outrigger canoe and pad...",N,White shark,6303,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,0,0,0,0
1,2018.06.18,18-Jun-2018,2018,14h00 -15h00,USA,Georgia,"St. Simon Island, Glynn County",Adyson McNeely,F,11,Unprovoked,Standing,Minor injury to left thigh,N,unknown,6302,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,0,0,0,0
2,2018.06.09,09-Jun-2018,2018,07h45,USA,Hawaii,"Habush, Oahu",John Denges,M,48,Invalid,Surfing,Injury to left lower leg from surfboard skeg,N,unknown,6301,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,0,0,0,0
3,2018.06.08,08-Jun-2018,2018,unknown,AUSTRALIA,New South Wales,Arrawarra Headland,male,M,unknown,Unprovoked,Surfing,Minor injury to lower leg,N,2 m shark,6300,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,0,0,0,0
4,2018.06.04,04-Jun-2018,2018,unknown,MEXICO,Colima,La Ticla,Gustavo Ramos,M,unknown,Provoked,Free diving,Lacerations to leg & hand shark PROVOKED INCIDENT,N,"Tiger shark, 3m",6299,A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,0,0,0,0


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6302 entries, 0 to 6301
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   case_number             6302 non-null   object
 1   date                    6302 non-null   object
 2   year                    6302 non-null   int32 
 3   time                    6302 non-null   object
 4   country                 6302 non-null   object
 5   area                    6302 non-null   object
 6   location                6302 non-null   object
 7   name                    6302 non-null   object
 8   sex                     6302 non-null   object
 9   age                     6302 non-null   object
 10  type                    6302 non-null   object
 11  activity                6302 non-null   object
 12  injury                  6302 non-null   object
 13  fatal                   6302 non-null   object
 14  species                 6302 non-null   object
 15  orig

### Normalizing activities column

Since my objective is to see which activity is the one prefered by each shark, species and activities are a priority

We're going to simplify each activity to one word only, so we're going to see which one word activities are already in the dataframe.

In [43]:
activities = []

for v in df.activity:
    lst = v.split()
    
    if len(lst) == 1:
        if lst[0].isalpha():          # Filter special characters out
            activities.append(lst[0])
        
activities = set(activities)

# activities

Some of them seem to be a subset or a derivation of another one (like `Murder` and `Murdered`) or just misspelled (like `Kakaying`).

In [44]:
# I used an external unknown web (chatgpt) to clean the set of activities

surfing_activities = {
    'Surfing',
    'Skimboarding',
    'Kiteboarding',
    'Kitesurfing',
    'Windsurfing',
    'Bodyboarding',
    'SUP',
    'Wakeboarding',
    'Surf'
}

fishing_activities = {
    'Angling',
    'Fisherman',
    'Fishing',
    'Clamming',
    'Crabbing',
    'Lobstering',
    'Oystering',
    'Shrimping',
    'Spearfishing'
}

bathing_activities = {
    'Bather',
    'Bathing',
    'Bath'
}

boating_activities = {
    'Boat',
    'Paddling',
    'Canoeing',
    'Rowing',
    'Sailing',
    'Kayaking',
    'Cruising',
    'Filming',
    'Hilo',
    'Sculling'
}

swimming_activities = {
    'Jumping',
    'Playing',
    'Swimming',
    'Splashing',
    'Swim'
}

diving_activities = {
    'Skindiving',
    'Snorkeling',
    'Diving'
}

other_activities = {
    'Crawling',
    'Rescuing',
    'Shipwreck',
    'Sitting',
    'Standing',
    'Suicide',
    'Unknown',
    'Wading',
    'Walking',
    'Washing'
}

In [45]:
# Fuzzywuzzy allows us to find similarity between strings in a percentile base. We're looking for 80% similarity.

def activity_category(x):
    
    '''
    Checks in which general category the activity is.
    
    '''

    x = x.lower()
    
    for i in surfing_activities:
        if fuzz.partial_ratio(i.lower(), x) > 80:
            return 'Surf'
        
    for i in fishing_activities:
        if fuzz.partial_ratio(i.lower(), x) > 80:
            return 'Fishing'

    for i in bathing_activities:
        if fuzz.partial_ratio(i.lower(), x) > 80:
            return 'Bathing'

    for i in swimming_activities:
        if fuzz.partial_ratio(i.lower(), x) > 80:
            return 'Swimming'

    for i in diving_activities:
        if fuzz.partial_ratio(i.lower(), x) > 80:
            return 'Diving'

    for i in boating_activities:
        if fuzz.partial_ratio(i.lower(), x) > 80:
            return 'Boating'

    return 'Other'

In [46]:
# we are going to use the unnamed_1 to put the general activity
df.rename(columns={'unnamed_1': 'activity_category'}, inplace=True)

# let's put activity_category before activity
new_col_order = ['case_number', 'date', 'year', 'time', 'country', 'area', 'location',
       'name', 'sex', 'age', 'type', 'activity_category', 'activity', 'injury', 'fatal', 'species',
       'original_order', 'investigator_or_source', 'pdf', 'href_formula',
       'href', 'unnamed_2', 'case_number_1',
       'case_number_2']

df = df[new_col_order]

# and apply activity_category function
df.activity_category = df.activity.apply(activity_category)

df.activity_category.value_counts()

Surf        1552
Other       1426
Fishing     1169
Swimming    1129
Diving       603
Bathing      222
Boating      201
Name: activity_category, dtype: int64

In [47]:
df.head()

,case_number,date,year,time,country,area,location,name,sex,age,type,activity_category,activity,injury,fatal,species,original_order,investigator_or_source,pdf,href_formula,href,unnamed_2,case_number_1,case_number_2
0,2018.06.25,25-Jun-2018,2018,18h00,USA,California,"Oceanside, San Diego County",Julie Wolfe,F,57,Boating,Boating,Paddling,"No injury to occupant, outrigger canoe and pad...",N,White shark,6303,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,0,0,0
1,2018.06.18,18-Jun-2018,2018,14h00 -15h00,USA,Georgia,"St. Simon Island, Glynn County",Adyson McNeely,F,11,Unprovoked,Other,Standing,Minor injury to left thigh,N,unknown,6302,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,0,0,0
2,2018.06.09,09-Jun-2018,2018,07h45,USA,Hawaii,"Habush, Oahu",John Denges,M,48,Invalid,Surf,Surfing,Injury to left lower leg from surfboard skeg,N,unknown,6301,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,0,0,0
3,2018.06.08,08-Jun-2018,2018,unknown,AUSTRALIA,New South Wales,Arrawarra Headland,male,M,unknown,Unprovoked,Surf,Surfing,Minor injury to lower leg,N,2 m shark,6300,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,0,0,0
4,2018.06.04,04-Jun-2018,2018,unknown,MEXICO,Colima,La Ticla,Gustavo Ramos,M,unknown,Provoked,Diving,Free diving,Lacerations to leg & hand shark PROVOKED INCIDENT,N,"Tiger shark, 3m",6299,A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,0,0,0


### Normalizing species column

We're going to follow more or less the same strategy as `activities`.

In [48]:
df.species.value_counts()

unknown                                                                                                                          2838
White shark                                                                                                                       163
Shark involvement prior to death was not confirmed                                                                                105
Invalid                                                                                                                           102
Shark involvement not confirmed                                                                                                    88
                                                                                                                                 ... 
1.2 m to 1.5 m [4.5' to 5'] shark                                                                                                   1
Bull shark, 2.3 m [7.5']                                      

Some of them appear to have the size of the shark, so a column is going to be added called "shark_size" with the shark size, but first we need to separate it.

In [49]:
lst = []

for i in df.species:
    
    if 'shark' in i.lower():
        splitted = i.lower().split()
        index = 0
        for i in splitted:
            if 'shark' in i:
                break
            else:
                index += 1
        if splitted[index -1].isalpha():
            lst.append(splitted[index - 1])
        
# set(lst)   

In [50]:
# as before, I used the unknown chatgpt to create a list of shark species
shark_species = {
    'angel',
    'barracua',
    'basking',
    'blacktip',
    'blue',
    'bluefish',
    'bonita',
    'bonnethed',
    'brown',
    'bull',
    'carcharinid',
    'carpet',
    'cookiecutter',
    'copper',
    'cow',
    'dusky',
    'galapagos',
    'goblin',
    'grey',
    'hammerhead',
    'lemon',
    'leopard',
    'mako',
    'nurse',
    'porbeagle',
    'raggedtooth',
    'reef',
    'salmon',
    'sandbar',
    'sandtiger',
    'shovelnose',
    'silky',
    'silvertip',
    'smoothhound',
    'soupfin',
    'spinner',
    'stingray',
    'thresher',
    'tiger',
    'wobbegong',
    'whale',
    'whaler',
    'white',
    'whitetip',
    'zambesi',
    'zambezi'
}

In [51]:
def fill_species(x):
    
    '''
    Categorize each shark.
    
    '''
    
    
    x = x.lower()
    for i in shark_species:
        i = i.lower()
        if fuzz.partial_ratio(i.lower(), x) > 80:
            return i.capitalize()
        
    return 'Other'

In [52]:
# We take two columns: one for the shark species and the other one for the size. The original shark we are going to keep it for now.

df.rename(columns={'species': 'shark', 'unnamed_2': 'shark_species', 'case_number_1': 'shark_size'}, inplace=True)

new_col_order = ['case_number', 'date', 'year', 'time', 'country', 'area', 'location',
       'name', 'sex', 'age', 'type', 'activity_category', 'activity', 'injury', 'fatal', 'shark_species', 'shark_size',
       'original_order', 'investigator_or_source', 'pdf', 'href_formula',
       'href', 'shark', 'case_number_2']

df = df[new_col_order]

# And apply the function
df.shark_species = df.shark.apply(fill_species)

# df.shark_species.value_counts()

In [53]:
df.head()

,case_number,date,year,time,country,area,location,name,sex,age,type,activity_category,activity,injury,fatal,shark_species,shark_size,original_order,investigator_or_source,pdf,href_formula,href,shark,case_number_2
0,2018.06.25,25-Jun-2018,2018,18h00,USA,California,"Oceanside, San Diego County",Julie Wolfe,F,57,Boating,Boating,Paddling,"No injury to occupant, outrigger canoe and pad...",N,White,0,6303,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,White shark,0
1,2018.06.18,18-Jun-2018,2018,14h00 -15h00,USA,Georgia,"St. Simon Island, Glynn County",Adyson McNeely,F,11,Unprovoked,Other,Standing,Minor injury to left thigh,N,Other,0,6302,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,unknown,0
2,2018.06.09,09-Jun-2018,2018,07h45,USA,Hawaii,"Habush, Oahu",John Denges,M,48,Invalid,Surf,Surfing,Injury to left lower leg from surfboard skeg,N,Other,0,6301,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,unknown,0
3,2018.06.08,08-Jun-2018,2018,unknown,AUSTRALIA,New South Wales,Arrawarra Headland,male,M,unknown,Unprovoked,Surf,Surfing,Minor injury to lower leg,N,Other,0,6300,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2 m shark,0
4,2018.06.04,04-Jun-2018,2018,unknown,MEXICO,Colima,La Ticla,Gustavo Ramos,M,unknown,Provoked,Diving,Free diving,Lacerations to leg & hand shark PROVOKED INCIDENT,N,Tiger,0,6299,A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,"Tiger shark, 3m",0


In [54]:
# now use regex to extract the size
pattern = r'(\d+(?:\.\d+)?)\s*m'

df.shark_size = df.shark.str.extract(pattern) + 'm'

df.shark_size.fillna('unknown', inplace=True)

df.head()

,case_number,date,year,time,country,area,location,name,sex,age,type,activity_category,activity,injury,fatal,shark_species,shark_size,original_order,investigator_or_source,pdf,href_formula,href,shark,case_number_2
0,2018.06.25,25-Jun-2018,2018,18h00,USA,California,"Oceanside, San Diego County",Julie Wolfe,F,57,Boating,Boating,Paddling,"No injury to occupant, outrigger canoe and pad...",N,White,unknown,6303,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,White shark,0
1,2018.06.18,18-Jun-2018,2018,14h00 -15h00,USA,Georgia,"St. Simon Island, Glynn County",Adyson McNeely,F,11,Unprovoked,Other,Standing,Minor injury to left thigh,N,Other,unknown,6302,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,unknown,0
2,2018.06.09,09-Jun-2018,2018,07h45,USA,Hawaii,"Habush, Oahu",John Denges,M,48,Invalid,Surf,Surfing,Injury to left lower leg from surfboard skeg,N,Other,unknown,6301,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,unknown,0
3,2018.06.08,08-Jun-2018,2018,unknown,AUSTRALIA,New South Wales,Arrawarra Headland,male,M,unknown,Unprovoked,Surf,Surfing,Minor injury to lower leg,N,Other,2m,6300,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2 m shark,0
4,2018.06.04,04-Jun-2018,2018,unknown,MEXICO,Colima,La Ticla,Gustavo Ramos,M,unknown,Provoked,Diving,Free diving,Lacerations to leg & hand shark PROVOKED INCIDENT,N,Tiger,3m,6299,A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,"Tiger shark, 3m",0


In [55]:
# and now we neutralize the shark column's values

df.shark = 0

Idea for the future: with more time, the country, area and even location could be used to identify which type of shark unknown species would probably be.

### Normalizing Time column

In [59]:
df.time.unique()[:10]

array(['18h00', '14h00  -15h00', '07h45', 'unknown', 'Late afternoon',
       '17h00', '14h00', 'Morning', '15h00', '08h15'], dtype=object)

There are some values that are the hour, others that are the time of the day and others that just seem random. We are going to take one of the unnamed column and call it "time of the day" to have a more general daytime such as "noon", "midday" and such. The original one, we're going to use it for the hours. The ones that have an exact time or a time lapse we're going to keep it like that, but those who have things like "Afternoon" we are going to assign a time-lapse that symbolizes, in that case, the afternoon

In [60]:
# Let's change the name of one of those placeholder columns and bring it next to 'time'

df.rename(columns={'shark': 'time_of_day'}, inplace=True)

new_col_order = ['case_number', 'date', 'year', 'time_of_day', 'time', 'country', 'area', 'location',
       'name', 'sex', 'age', 'type', 'activity_category', 'activity', 'injury', 'fatal', 'shark_species', 'shark_size',
       'original_order', 'investigator_or_source', 'pdf', 'href_formula',
       'href', 'case_number_2']

df = df[new_col_order]


In [62]:
df.head(1)

,case_number,date,year,time_of_day,time,country,area,location,name,sex,age,type,activity_category,activity,injury,fatal,shark_species,shark_size,original_order,investigator_or_source,pdf,href_formula,href,case_number_2
0,2018.06.25,25-Jun-2018,2018,0,18h00,USA,California,"Oceanside, San Diego County",Julie Wolfe,F,57,Boating,Boating,Paddling,"No injury to occupant, outrigger canoe and pad...",N,White,unknown,6303,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,0


In [127]:
# first, let's create a dictionary with time of day and the ranges of that time of day

def which_time(x):
    
    if x == 'unknown':
        return x
    
    try:
        x = int(x.lower().split('h')[0])
        
        if x > 23:                 # for values such as '1600' that should be '16:00'
            x = int(x / 100)
        
        time_ranges = {
            'Morning': (6, 11),    # '06:00', '11:59'
            'Afternoon': (12, 17), # '12:00', '17:59'
            'Evening': (18, 21),   # '18:00', '21:59'
        }
     
        for t_range, (start, end) in time_ranges.items():
            if start <= x <= end:
                return t_range
        
        # range for night is different because it start with a large number (22) and ends with a small one (5)
        
        if (22 <= x <= 24) | (0 <= x <= 5):
            return 'Night'
        
        return None

    except:
        time_ranges = ['Morning', 'Afternoon', 'Evening', 'Night']

        for t_range in time_ranges:
            if fuzz.partial_ratio(t_range, x) > 80:
                return t_range
           
        return None

In [128]:
df.time_of_day = df.time.apply(which_time)

# test.head(10)

In [129]:
napercent()

# some na, let's see

time_of_day    1.904157
dtype: float64

In [136]:
df[df.time_of_day.isnull() == True].head()

,case_number,date,year,time_of_day,time,country,area,location,name,sex,age,type,activity_category,activity,injury,fatal,shark_species,shark_size,original_order,investigator_or_source,pdf,href_formula,href,case_number_2
98,2017.08.26,26-Aug-2017,2017,None,Midday,SPAIN,Castellón,Grao de Moncofa,female,F,11,Invalid,Swimming,Swimming,Lacerations to left foot,unknown,Blue,unknown,6205,"El Periodico Mediterraneo, 8/27/2017",2017.08.26.b-Spain.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,0
138,2017.06.02,02-Jun-2017,2017,None,Shortly before 12h00,BAHAMAS,New Providence,Athol Island,Tiffany Johnson,F,32,Unprovoked,Diving,Snorkeling,Right forearm severed,N,Tiger,unknown,6165,"Tribune 242, 6/2/2017",2017.06.02-Johnson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,0
301,2016.03.03,Reported 03-Mar-2016,2016,None,Midnight,AUSTRALIA,South Australia,Wrights Bay,Lee Taplin,M,unknown,Unprovoked,Fishing,Fishing,Puncture wounds to right calf,N,Whale,unknown,6002,"9 News, 3/1/2016",2016.03.03.R-Taplin.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,0
472,2014.11.29,29-Nov-2014,2014,None,Sometime between 06h00 & 08hoo,AUSTRALIA,Western Australia,"Pyramids Beach, Port Bouvard",Cameron Pearman,M,13,Unprovoked,Surf,Surfing,Minor injuries to right leg,N,Other,unknown,5831,"The Sydney Morning Herald, 11/29/2014",2014.11.29-Pearman.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,0
743,2012.09.06,06-Sep-2012,2012,None,Just before noon,USA,Florida,"Neptune Beach, Duval County",James Fyfe,M,30s,Unprovoked,Surf,Surfing,Right calf bitten,N,Other,unknown,5560,"New4Jax, 9/7/2012",2012.09.06.b-Fyfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,0


Fixing the null values generated would be too much work for the time I have, so I'm going to drop the 120 values. With more time and knowledge I would probably use an NLP pipeline to analize each of the remaining values to categorize them better, but for now since dropping them leave me with more than 2500 row, I'm going to do just that.

In [138]:
df.dropna(inplace=True)

napercent()

Series([], dtype: float64)

In [139]:
df.shape, df_ori.shape

((6182, 24), (25723, 24))

Now, to clean the `time` column itself. The idea is to keep hours values and, if there are no hour values, assign a time range depending on time_of_day.

## Objective

My objective is to retrieve how each species of shark like their food, so I'm going to see what is the most repeated activity each victim was doing before being eaten by a shark.

In [97]:
df.species.value_counts()

unknown                                                                                                                          2838
White shark                                                                                                                       163
Shark involvement prior to death was not confirmed                                                                                105
Invalid                                                                                                                           102
Shark involvement not confirmed                                                                                                    88
                                                                                                                                 ... 
1.2 m to 1.5 m [4.5' to 5'] shark                                                                                                   1
Bull shark, 2.3 m [7.5']                                      

In [83]:
df.activity

0           Paddling
1           Standing
2            Surfing
3            Surfing
4        Free diving
            ...     
6297          Diving
6298    Pearl diving
6299        Swimming
6300         unknown
6301        Swimming
Name: activity, Length: 6302, dtype: object